In this file we will check Accuracy of each Tool of sentiment against manually anotated data:
1) Vader
2) ROBERTa
3) FinBert

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report

# Download the VADER lexicon (only needed once)
nltk.download("vader_lexicon")

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Define a function to get sentiment label and compound score using VADER
def get_vader_sentiment(text):
    # If text is not a valid string or is empty, return Neutral sentiment
    if not isinstance(text, str) or text.strip() == "":
        return "Neutral", 0.0
    
    scores = sia.polarity_scores(text)
    compound = scores["compound"]
    
    # Use standard VADER thresholds to assign a label
    if compound >= 0.05:
        label = "Positive"
    elif compound <= -0.05:
        label = "Negative"
    else:
        label = "Neutral"
    
    return label, compound

# Load your test dataset (ensure it has 'text' and 'sentiment_numeric' columns)
# test_df = pd.read_csv('/notebooks/test_data.csv')

# Ensure the 'text' column contains valid strings
test_df['text'] = test_df['text'].fillna('').astype(str)

# Drop rows with missing ground truth sentiment_numeric values
test_df = test_df.dropna(subset=['sentiment_numeric'])

# Apply VADER sentiment analysis on the 'text' column
# This creates two new columns: one for the sentiment label and one for the compound score
test_df["VADER_Sentiment"], test_df["VADER_Score"] = zip(*test_df["text"].apply(get_vader_sentiment))

# Map VADER sentiment labels to numeric values:
# "Negative" -> -1.0, "Neutral" -> 0.0, "Positive" -> 1.0
label_to_numeric = {"Negative": -1.0, "Neutral": 0.0, "Positive": 1.0}
test_df["VADER_Sentiment_Numeric"] = test_df["VADER_Sentiment"].map(label_to_numeric)

# Evaluate the VADER predictions against the ground truth
true_numeric = test_df["sentiment_numeric"].tolist()
pred_numeric = test_df["VADER_Sentiment_Numeric"].tolist()

accuracy = accuracy_score(true_numeric, pred_numeric)
print("Accuracy (numeric):", accuracy)
print("\nClassification Report (numeric):")
print(classification_report(true_numeric, pred_numeric))


In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

# ✅ Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# ✅ Load Roberta-based Sentiment Model & Tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name).to(device)
model.eval()  # Set model to evaluation mode

# ✅ Load Test Data
# Ensure your test CSV contains both 'text' and 'sentiment_numeric' columns.
# test_df = pd.read_csv('/notebooks/test_data.csv')
# Replace missing text with an empty string and ensure all entries are strings.
test_df['text'] = test_df['text'].fillna('').astype(str)
# Drop rows with missing sentiment_numeric ground truth.
test_df = test_df.dropna(subset=['sentiment_numeric'])

# Convert the text column to a list of strings (this ensures proper type for tokenization)
test_texts = [str(text) for text in test_df['text'].tolist()]

# ✅ Define Batch Processing Function for Roberta
def get_roberta_sentiment_scores(texts, batch_size=32):
    """Process texts in batches and return predicted class indices."""
    predictions = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing Sentiment", unit="batch", ncols=80):
        batch_texts = texts[i:i + batch_size]
        # Tokenize and convert texts to input tensors
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
        # Move tensors to the appropriate device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Apply softmax to get probabilities and then get the predicted class indices
        probs = F.softmax(outputs.logits, dim=-1)
        batch_preds = torch.argmax(probs, dim=-1).cpu().numpy()
        predictions.extend(batch_preds)
    
    return predictions

# ✅ Get predictions on test texts
predicted_class_indices = get_roberta_sentiment_scores(test_texts)

# Mapping: the model outputs classes 0, 1, 2 which correspond to:
# 0 -> negative, 1 -> neutral, 2 -> positive.
# We map these to your numeric labels: negative (-1.0), neutral (0.0), positive (1.0)
index_to_numeric = {0: -1.0, 1: 0.0, 2: 1.0}
predicted_numeric = [index_to_numeric[idx] for idx in predicted_class_indices]

# ✅ Evaluate using the ground truth 'sentiment_numeric'
true_numeric = test_df['sentiment_numeric'].tolist()

accuracy = accuracy_score(true_numeric, predicted_numeric)
print("Accuracy (numeric):", accuracy)
print("\nClassification Report (numeric):")
print(classification_report(true_numeric, predicted_numeric))

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

# Load the test dataset (ensure it contains the tweet text)

# If the text column exists but might have missing values, fill them
test_df['text'] = test_df['text'].fillna('').astype(str)

# Drop rows where sentiment_numeric is missing
test_df = test_df.dropna(subset=['sentiment_numeric'])

# Process tweet texts to obtain sentiment scores from FinBERT
test_texts = test_df['text'].tolist()
sentiment_scores = get_finbert_sentiment_score(test_texts)

# Convert continuous sentiment scores into discrete string labels
def score_to_label(score):
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

predicted_labels = [score_to_label(score) for score in sentiment_scores]

# Mapping: Adjusted to match your sentiment_numeric values
# negative -> -1.0, neutral -> 0.0, positive -> 1.0
label_to_numeric = {'negative': -1.0, 'neutral': 0.0, 'positive': 1.0}
predicted_numeric = [label_to_numeric[label] for label in predicted_labels]

# Ground truth numeric labels from the test file
true_numeric = test_df['sentiment_numeric'].tolist()

# Evaluate the predictions
accuracy = accuracy_score(true_numeric, predicted_numeric)
print("Accuracy (numeric):", accuracy)
print("\nClassification Report (numeric):")
print(classification_report(true_numeric, predicted_numeric))
